![NVIDIA Logo](images/nvidia.png)

# Responding in Your Own Voice

In this final notebook you will refactor your `respond_to_email` function to automatically reply to customer emails in a distinct voice.

---

## Learning Objectives

By the time you complete this notebook you will:
- Learn about a tool to help you train models to respond in your own distinct voice
- Be able to modify LLM responses to be in a distinct voice
- Reflect on everything you've learned over the course of this workshop

## Imports

In [1]:
import json

from tqdm.notebook import tqdm

from llm_utils.nemo_service_models import NemoServiceBaseModel
from llm_utils.helpers import collect_my_prompts_and_responses
from llm_utils.models import LoraModels
from llm_utils.postprocessors import strip
from llm_utils.llm_functions import (
    make_llm_function,
    get_sentiment,
    extract_name,
    extract_product,
    extract_location,
    generate_customer_response_email
)

---

## Models

In [2]:
LoraModels.list_models()

gpt8b: gpt-8b-000-lora
gpt43b: gpt-43b-002-lora


## Load Customer Emails

In [3]:
with open('data/solution_emails.json', 'r') as f:
    customer_emails = json.load(f)

---

# Responding in Your Own Voice

As a very last step in our task to autogenerate customer emails, we would like our email responses to capture our own voice and style of writing. A fantastic way to accomplish this is to fine-tune a model on text data that you have have written yourself in a natural setting.

As we've learned, fine-tuning can work quite well with ~1000 samples, and sometimes less. With this in mind you can either sit down and write responses to several hundred to a thousand "prompts", which you could curate and/or synthetically generate yourself, or, you could make it a point as you go about your day having hundreds of natural text interactions with others to collect "your prompts and responses".

The following opens a simple web page where you can submit "your prompts and responses" and saves them to `my_prompts_and_responses.jsonl` formatted correctly for NeMo Service p-tuning or LoRA.

As always, be responsible and take care not to send data to 3rd party services when you should not.

In [4]:
collect_my_prompts_and_responses()

You could even easily imagine adding some additional logic that uploaded the data and kicked off a fine-tuning job once your data got to a size you were looking for.

---

## Responding With a Pirate's Voice

But since we don't have a bunch of data of your voice, and since if we provided you data of how we sounded you might not really be able to tell the difference, we've provided you with a model that has been LoRA fine-tuned on pirate responses. We call this, our final PEFT technique of the workshop, **P**irate-**tuning**. 🏴‍☠️

![Persona Create](images/persona_create.png)

Continuing to leverage the virtuous cycle we've been discussing throughout the day, we generated our synthetic pirate response data by 2-shot prompting GPT43B to repeat the emails generated in the previous notebook (though we generated 700 instead of 50, since we had the time) in the voice of a pirate.

We passed the original (synthetically generated) emails into the following prompt format:

In [5]:
def pirate_template(statement):
    return f'Statement: {statement}Pirate Statement: '

...then saved GPT43B's responses as labels. We used the email/pirate-label pairs to perform LoRA for 1 epoch using GPT8B.

In [6]:
pirate_model = NemoServiceBaseModel(LoraModels.gpt8b.value, customization_id='2d03bab3-0d79-44ff-b21f-bb6cea86b4fc')

---

## Pirate LLM Function

![Persona LLM Function](images/persona_llm_function.png)

Here we build a `pirate` LLM function using the LoRA fine-tuned GPT8B model and the `pirate_template`.

In [7]:
pirate = make_llm_function(pirate_model, pirate_template, postprocessor=strip)

---

## Try Pirate Voice

In [8]:
prompt = """All of the sudden I know a lot about how to customize and orchestrate customized LLMs for fun and profit. \
Thank you prompt engineering. Thank you LoRA. Thank you NeMo."""

In [9]:
pirate(prompt, stop=['\n'])

"Arrrr, me hearty! I be havin' a grand time learnin' about how to customize and orchestrate customized LLMs for fun and profit, savvy? Thank ye, prompt engineering! Thank ye, LoRA! Thank ye, NeMo!"

---

## Auto Generate Response Emails in "Your Own" Voice.

In [10]:
def respond_to_email(email):
    name = name_extractor(email)
    sentiment = get_sentiment(email, tokens_to_generate=1)
    product = product_extractor(email)
    location = location_extractor(email)

    response = write_response_email(company_name, name, sentiment, product, location)
    return response

### Solution

In [11]:
def respond_to_email(email):
    name = extract_name(email)
    sentiment = get_sentiment(email, tokens_to_generate=1)
    product = extract_product(email)
    location = extract_location(email)

    company_name = 'StarBikes'
    response = generate_customer_response_email(company_name, name, sentiment, product, location)
    return pirate(response)

In [12]:
for customer_email in customer_emails[:1]:
    print(customer_email+'\n\n')
    print(respond_to_email(customer_email)+'\n---\n')

Hi,
I bought a fat tire bike from StarBikes in Los Angeles. The bike has been a nightmare for me. I am a big guy and I need something that can handle my weight and the bike just isn't doing it. I am very disappointed with my purchase and I would like to get a full refund. Please contact me as soon as you can.
Thanks,
William


Arrrr, me hearty!

Ahoy, William!

Thank ye for yer email, matey! We be sorry to hear about yer experience with yer fat tire bike, but we be happy to help ye out, savvy? Someone from our store in Los Angeles will be contacting ye soon to discuss the matter in more detail, and we'll do our best to make it right, me hearties!

Fair winds and following seas,
StarBikes Customer Support Team
---



---